## Scope
In this notebook we give a very brief tutorial which focuses on the mnms python user interface. We'll use a highly downgraded version of the `pa5a` data so that our operations are fast, even though you will probably never use that sim. This way you know how to use mnms in your own analysis code.

## Tutorial

In mnms, we always need to generate a "sqrt-covariance" matrix as a first step; only then can we generate sims:

In [ ]:
from mnms import noise_models as nm, utils
from soapack import interfaces as sints

# load the DR5 data model in order to be able to use deep6 (qid = 'd6') data
dm = sints.DR6v3()

Create a `TiledNoiseModel` instance. By supplying `downgrade=8` and no other `lmax` information, the `lmax` of this model will be set to the Nyquist frequency of the pixels, 2700:

In [ ]:
tnm = nm.TiledNoiseModel('pa5a', data_model=dm, downgrade=8, mask_version='v3',
                         mask_est_name='dr6v3_20220316_baseline_union_mask', notes='my_first_model')

Now we need to make our noise model. Because they are the slowest step to make, and take up a lot of space, by default they are written to disk. The idea is that outside of testing/development, you only need to ever run this command once (per data-release, array-set, other parameters, etc). As you can see, a lot of the time in the below cell is spent loading and preparing the metadata.

This cell will take a couple minutes (if it only takes a few seconds, it's probably because someone has already run this notebook and saved models into the same directory. Just change the `notes` kwargs to something unique and rerun the above cell):

In [ ]:
_ = tnm.get_model(0, keep_model=True, keep_ivar=True, keep_dmap=True)

Now make a `WaveletNoiseModel` instance. To save time, we'll pass some of the metadata loaded from the first model into the second model:

In [ ]:
wnm = nm.WaveletNoiseModel('pa5a', data_model=dm, downgrade=8, mask_version='v3', 
                           mask_est_name='dr6v3_20220316_baseline_union_mask', notes='my_first_model',
                           mask_est=tnm.mask_est, mask_obs=tnm.mask_obs, ivar_dict=tnm.ivar_dict,
                           dmap_dict=tnm.dmap_dict)

And make the noise model again (same thing as before -- if this takes only a few seconds, change the `notes` kwarg and rerun the above cell first):

In [ ]:
_ = wnm.get_model(0, keep_model=True, keep_ivar=True)

To save you from accidentally regenerating the exact same model again, the argument `check_on_disk` is `True` by default. If I rerun `get_model`, it will find that the model is on-disk, and return it in a dictionary (you can also keep the model in memory, stored as an instance attribute, if you want, via `keep_model=True`, which we did in the previous cells):

In [ ]:
_ = tnm.get_model(0)

In [ ]:
_ = wnm.get_model(0)

That was fast, and that's because the models are on-disk. Let's look at my `covmat_path` from the `mnms` block of my soapack config (if you changed the `notes` in your `NoiseModel`s, replace that search string below):

In [ ]:
ls /scratch/gpfs/zatkins/data/ACTCollaboration/mnms/covmats/*my_first_model*

Note, this trick won't work if you give a model a different `notes` parameter. E.g., `my_second_model` is considered a totally distinct noise model from `my_first_model`, even if all the "scientific" parameters are the same.

Now we can make sims. To do so, we must load the "sqrt-covariance" matrix from disk, so that we can sample from it, so make sure to have enough memory on-hand to do this. These files are only loaded once by default using the `keep_model` kwarg -- they are stored in the object instance, so that future calls to `get_sim` do not have to load from disk. There is also a similar `check_on_disk` parameter to prevent users from generating the same sim twice (by default this is `True`).

By default, these are ***not*** saved to disk (but they can be, if desired). We need to tell `get_sim` which split and map number we want to make. The map number is used in getting a unique random seed (and if the sims is written to disk, will be stored in the filename):

In [ ]:
talm_s0_n123 = tnm.get_sim(0, 123, alm=True, verbose=True)

In [ ]:
walm_s0_n123 = wnm.get_sim(0, 123, alm=True, verbose=True)

As noted in the README, these sims always have shape (num_arrays, num_splits, num_pol, ny, nx), even if some of these axes have dimension 1. Because we generate sims per split, `num_splits=1` always. Let's take a look! 

In [ ]:
tmap_s0_n123 = utils.alm2map(talm_s0_n123, shape=tnm.shape, wcs=tnm.wcs)
wmap_s0_n123 = utils.alm2map(walm_s0_n123, shape=wnm.shape, wcs=wnm.wcs)

utils.eshow(tmap_s0_n123, colorbar=True, downgrade=8, ticks=30)
utils.eshow(wmap_s0_n123, colorbar=True, downgrade=8, ticks=30)

Finally, we can take our two models and stitch their sims together in harmonic space. We'll have a transition region from ell=1300 to ell=1500, with a cosine profile. We'll let the `WaveletNoiseModel` be the low-ell model by ordering it first when we build a `HarmonicMixture` object:

In [ ]:
hm = nm.HarmonicMixture([wnm, tnm], [1400], [200], profile='cosine')
stitched_sim_s0_n123 = hm.get_sim(0, 123, alm=True, verbose=True)

Had we saved our original sims to disk, the above cell would have run even faster. By default (since `check_on_disk` is `True`), the call would have loaded the sims from disk rather than generating them on-the-fly. Anyway, we can take a look:

In [ ]:
print(stitched_sim_s0_n123.shape, stitched_sim_s0_n123.dtype)

stitched_sim_s0_n123 = utils.alm2map(stitched_sim_s0_n123, shape=hm.shape, wcs=hm.wcs)
utils.eshow(stitched_sim_s0_n123, colorbar=True, downgrade=8, ticks=30)